In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/baihaocheng/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

We are using the following features
- TfIdf score
- topic frequency
- emoticons

## tfidf



am, is, are, ---> be. 

In [2]:
train_csv_path = '../data/development.csv'
train_df = pd.read_csv(train_csv_path,index_col=0)

In [3]:
train_df

,body,type,body len,body char len,subreddit
95748,Be open-minded and genuinely interested in oth...,ENFJ,138,730,enfj
626910,ENFJs need to realize that to take care of oth...,ENFJ,207,1153,enfj
75943,I know people that are non-pretentious hipster...,ENFJ,213,1264,enfj
335326,My INTJ partner was trying to help me with col...,ENFJ,142,761,intj
1387161,"Personally, I'm not picky. Really any physical...",ENFJ,69,362,INTP
...,...,...,...,...,...
1325016,this is all true. it's an assumption on my par...,ISTP,362,2047,istp
1747454,lol you guys aren't stereotyped as blind to th...,ISTP,59,352,mbti
83834,I'm around women the same way I'm around men. ...,ISTP,191,1017,istp
553224,"&gt;I've lost a hundred pounds, but I did it f...",ISTP,95,518,INTP


In [6]:
def clean(sentence):
    # lemmetize,
    # stopwords
    # handling some words more subtlely
    stop_words = stopwords.words("english")
    sentence = sentence.lower().encode('ascii', 'ignore').decode()
    sentence = ' '.join([word for word in sentence.split(' ') if word not in stop_words])
    sentence = re.sub("@\S+", " ", sentence)
    sentence = re.sub("https*\S+", " ", sentence)
    sentence = re.sub("#\S+", " ", sentence)
    sentence = re.sub("\'\w+", '', sentence)
    sentence = re.sub('[%s]' % re.escape(string.punctuation), ' ', sentence)
    sentence= re.sub(r'\w*\d+\w*', '', sentence)
    sentence = re.sub('\s{2,}', " ", sentence)
    return sentence
    
train_df['body'] = train_df['body'].apply(clean)
train_df['body']

95748      open minded genuinely interested people think ...
626910     enfjs need realize take care others need take ...
75943      know people non pretentious hipsters exist mea...
335326     intj partner trying help college algebra one t...
1387161    personally i picky really physical verbal affe...
                                 ...                        
1325016    true assumption part assume individuals field ...
1747454    lol guys stereotyped blind damage inflict ster...
83834      i around women way i around men sense level in...
553224      gt i lost hundred pounds me this truth someon...
593954      gt introverts see us extroverts get energized...
Name: body, Length: 1600, dtype: object

In [7]:
for sentence in train_df['body']:
    print(sentence)
    break

open minded genuinely interested people think things like i already enough friends people life close off take initiative wait people talk first call first etc as moving new city try checking meetups similar interests joining clubs volunteering taking classes go gym sit around cafe hang park anywhere others be you know could even post ad area asking someone tour guide one friends tour guide girl new city getting paid anything completely platonic fun go explore new city 


In [8]:
vectornizer = CountVectorizer()  
x = vectornizer.fit_transform(train_df['body'])

0

In [10]:
transformer = TfidfTransformer()  
tfidf = transformer.fit_transform(x)  
tfidf.toarray()


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
# tfidf_model = TfidfTransformer()
# tfidf = tfidf_model.fit_transform(X)